In [ ]:
import joblib
import pandas as pd
import os
import sys
from flask import Flask
from flask_restx import Api, Resource, fields, reqparse
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

def predict_m(duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature):

    clf = joblib.load(os.path.dirname(__file__) + '/train_music_P2.pkl') 
    #clf = joblib.load('train_music_P2.pkl')
    
    datatrack={'duration_ms':[duration_ms],
          'explicit':[explicit],
          'danceability':[danceability],
          'energy':[energy],
          'key':[key],
          'loudness':[loudness],
          'mode':[mode],
          'speechiness':[speechiness],
          'acousticness':[acousticness],
          'instrumentalness':[instrumentalness],
          'liveness':[liveness],
          'valence':[valence],
          'tempo':[tempo],
          'time_signature':[time_signature]}
    datatrack = pd.DataFrame(datatrack)

    # Make prediction
    p1 = clf.predict(datatrack)

    return p1


app = Flask(__name__)

# Definición API Flask
api = Api(
    app, 
    version='1.0', 
    title='Popularity track Prediction API',
    description='Popularity track prediction API')

ns = api.namespace('predict', 
     description='Regression to find the Popularity of a Song')

# Definición argumentos o parámetros de la API
parser = reqparse.RequestParser()

parser.add_argument(
    'duration_ms', 
    type=int, 
    required=True, 
    help='Duration of a song in ms', 
    location='args')
parser.add_argument(
    'explicit', 
    type=int, 
    required=True, 
    help='Is 1 if the song have a explicit lirycs', 
    location='args')
parser.add_argument(
    'danceability', 
    type=float, 
    required=True, 
    help='A value of 0.0 is less danceable and 1.0 is totally danceable.', 
    location='args')
parser.add_argument(
    'energy', 
    type=float, 
    required=True, 
    help='A value between 0.0 and 1.0 that represent the intensity and perceptual activity of the track', 
    location='args')
parser.add_argument(
    'key', 
    type=int, 
    required=True, 
    help='The key in which the track is located', 
    location='args')
parser.add_argument(
    'loudness', 
    type=float, 
    required=True, 
    help='The overall loudness of the track in decibels', 
    location='args')
parser.add_argument(
    'mode', 
    type=int, 
    required=True, 
    help='Indicates the mode (major or minor) of the track, 1: major, 0:minor', 
    location='args')
parser.add_argument(
    'speechiness', 
    type=float, 
    required=True, 
    help='Detects the presence of spoken words in the track. 1 indicate that the track is entirely spoken and 0.33 indicate a mix between music and speak.', 
    location='args')
parser.add_argument(
    'acousticness', 
    type=float, 
    required=True, 
    help='A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1 is a high confidence that the track is acoustic. ', 
    location='args')
parser.add_argument(
    'instrumentalness', 
    type=float, 
    required=True, 
    help='Predicts if a track contains no vocals. 1 means that the track have a high probability to be instrumental', 
    location='args')
parser.add_argument(
    'liveness', 
    type=float, 
    required=True, 
    help='Detects the presence of an audience in the recording. 1 means that the track is live record.', 
    location='args')
parser.add_argument(
    'valence', 
    type=float, 
    required=True, 
    help='A measure from 0 to 1 that describes the musical positivity of the track, high values mean a positive track', 
    location='args')
parser.add_argument(
    'tempo', 
    type=float, 
    required=True, 
    help='The estimated tempo of the track in beats per minute (BPM)', 
    location='args')
parser.add_argument(
    'time_signature', 
    type=int, 
    required=True, 
    help='An estimated time signature, indicating how many beats there are in each measure. Values between 3 to 7 that means 3/4, 4/4...', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})


@ns.route('/')
class PopularityTrackApi(Resource):
    @ns.expect(parser)
    #@ns.doc(parser=parser)
    @ns.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()

        try: 
            prediction = predict_m(args['duration_ms'],
            args['explicit'],
            args['danceability'],
            args['energy'],
            args['key'],
            args['loudness'],
            args['mode'],
            args['speechiness'],
            args['acousticness'],
            args['instrumentalness'],
            args['liveness'],
            args['valence'],
            args['tempo'],
            args['time_signature'])
            return {"result":prediction}, 200
        except Exception as e:
            return {'error': str(e)}, 400


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)